# Noisy-And MIL

Now we're going to load and train and MIL network on MNIST (presence of 0 = positive bag).

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from noisy_and import NoisyAnd
import mil_benchmarks

N_CLASSES = 2
MAX_BAG = 7

utils.gpu_fix()

tf.random.set_seed(42)
print(tf.__version__)

In [ ]:
def define_model(shape=(28, 28, 1)):
  model = tf.keras.Sequential([
    layers.Input((MAX_BAG,) + shape),
    *utils.baseline_layers(shape, N_CLASSES),
    NoisyAnd(N_CLASSES),
  ])

  utils.compile(model, N_CLASSES)
  return model

print(define_model().summary())

In [ ]:
standard_mnist = utils.evaluate_all(define_model, 'Noisy-And', mil_benchmarks.standard.mnist)
utils.plot_histories(standard_mnist, title='Noisy-And / Standard Fashion', filename='img/noisyand-standard-mnist.jpg')

In [ ]:
standard_fashion = utils.evaluate_all(define_model, 'Noisy-And', mil_benchmarks.standard.fashion)
utils.plot_histories(standard_fashion, title='Noisy-And / Standard Fashion', filename='img/noisyand-standard-fashion.jpg')

In [ ]:
standard_cifar10 = utils.evaluate_all(define_model, 'Noisy-And', mil_benchmarks.standard.cifar10, (32, 32, 3))
utils.plot_histories(standard_cifar10, title='Noisy-And / Standard CIFAR-10', filename='img/noisyand-standard-cifar10.jpg')

In [ ]:
presence_mnist = utils.evaluate_all(define_model, 'Noisy-And', mil_benchmarks.presence.mnist)
utils.plot_histories(presence_mnist, title='Noisy-And / Presence MNIST', filename='img/noisyand-presence-mnist.jpg')

In [ ]:
presence_fashion = utils.evaluate_all(define_model, 'Noisy-And', mil_benchmarks.presence.fashion)
utils.plot_histories(presence_fashion, title='Noisy-And / Presence Fashion', filename='img/noisyand-presence-fashion.jpg')

In [ ]:
presence_cifar10 = utils.evaluate_all(define_model, 'Noisy-And', mil_benchmarks.presence.cifar10, (32, 32, 3))
utils.plot_histories(presence_cifar10, title='Noisy-And / Presence CIFAR-10', filename='img/noisyand-presence-cifar10.jpg')

In [ ]:
absence_mnist = utils.evaluate_all(define_model, 'Noisy-And', mil_benchmarks.absence.mnist)
utils.plot_histories(absence_mnist, title='Noisy-And / Absence MNIST', filename='img/noisyand-absence-mnist.jpg')

In [ ]:
absence_fashion = utils.evaluate_all(define_model, 'Noisy-And', mil_benchmarks.absence.fashion)
utils.plot_histories(absence_fashion, title='Noisy-And / Absence Fashion', filename='img/noisyand-absence-fashion.jpg')

In [ ]:
absence_cifar10 = utils.evaluate_all(define_model, 'Noisy-And', mil_benchmarks.absence.cifar10, (32, 32, 3))
utils.plot_histories(absence_cifar10, title='Noisy-And / Absence CIFAR-10', filename='img/noisyand-absence-cifar10.jpg')

In [ ]:
complex_fashion = utils.evaluate_all(define_model, 'Noisy-And', mil_benchmarks.complex.fashion)
utils.plot_histories(complex_fashion, title='Noisy-And / Complex Fashion', filename='img/noisyand-complex-fashion.jpg')